In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [22]:
import mlflow

In [2]:
import pickle

In [3]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [4]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [5]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [6]:
categorical = ['PULocationID', 'DOLocationID']

In [7]:
df[categorical] = df[categorical].astype(str)

In [8]:
train_dicts = df[categorical].to_dict(orient='records')

In [9]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [10]:
target = 'duration'
y_train = df[target].values

In [11]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].astype('str')
    
    return df

In [12]:
df_val = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [13]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [14]:
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values

In [15]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

[0]	validation-rmse:7.66725                           
[1]	validation-rmse:6.06430                           
[2]	validation-rmse:5.84835                           
[3]	validation-rmse:5.72312                           
[4]	validation-rmse:5.50381                           
[5]	validation-rmse:5.47696                           
[6]	validation-rmse:5.44871                           
[7]	validation-rmse:5.38953                           
[8]	validation-rmse:5.35146                           
[9]	validation-rmse:5.33510                           
[0]	validation-rmse:8.43125                                                    
[1]	validation-rmse:7.12437                                                    
[2]	validation-rmse:6.75361                                                    
[3]	validation-rmse:6.55328                                                    
[4]	validation-rmse:6.44187                                                    
[5]	validation-rmse:6.00747                       